In [ ]:
###### Config #####
import sys, os, platform
if os.path.isdir("ds-assets"):
  !cd ds-assets && git pull
else:
  !git clone https://github.com/lutzhamel/ds-assets.git
colab = True if 'google.colab' in os.sys.modules else False
system = platform.system() # "Windows", "Linux", "Darwin"
home = "ds-assets/assets/"
sys.path.append(home)

In [ ]:
import pandas as pd
import dsutils
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()
from sklearn.datasets import make_blobs
from sklearn.datasets import make_circles
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [ ]:
# aux function
def plot_svm_surface(clf,X,surface_only=False,support_only=False):
  ax = plt.gca()
  if not support_only:
    DecisionBoundaryDisplay.from_estimator(
        clf,
        X,
        plot_method="contour",
        colors="k",
        levels= [0] if surface_only else [-1, 0, 1],
        alpha=0.5,
        linestyles= ["-"] if surface_only else ["--", "-", "--"],
        ax=ax,
    )
  # plot support vectors
  if (not surface_only) or support_only:
    ax.scatter(
        clf.support_vectors_[:, 0],
        clf.support_vectors_[:, 1],
        s=100,
        linewidth=1,
        facecolors="none",
        edgecolors="k",
    )

In [ ]:
# aux function
def plot_perceptron_surface(clf, X):
  ax = plt.gca()
  DecisionBoundaryDisplay.from_estimator(
      clf,
      X,
      plot_method="contour",
      levels = [0],
      colors="g",
      alpha=0.5,
      linestyles= '-',
      ax=ax,
  )

In [ ]:
# example data
def make_data():
   X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)
   return X, y


In [ ]:
# plot example data
def plot_data(X, y):
  plt.scatter(X[:, 0], X[:, 1], c=y, cmap='autumn')
  plt.xlabel('Feature 1')
  plt.ylabel('Feature 2')
  plt.title('Example Data')


# Large Margin Classifiers

Large margin classifiers are a **refinement of perceptron learning** in that they construct **optimal decision surfaces** that can discriminate between two classes.


Recall that the perceptron training algorithm keeps modifying the weights and the bias term of the perceptron architecture until a decision surface is found that separates the two classes,

<center>
<img src="https://raw.githubusercontent.com/lutzhamel/ds-assets/main/assets/perceptron-train.jpg"  height="250" width="280">
</center>


The problem with the learning heuristic for perceptrons is that it can give rise to **suboptimal decision surfaces**.  Perhaps something like this,

<center>
<img 
src="https://gamedevacademy.org/wp-content/uploads/2017/10/SVM-Perceptron-Decision-Boundary.png.webp"
width="350"
height="300">
</center>

We call this a **suboptimal decision surface**  because it has a **higher likelihood to misclassify points** bacause of the proximity to one of the classes.

As we will see large margin classifiers will construct decision surfaces that minimize misclassification due to proximity -- that is, large margin classifiers try to find decision surfaces that sit in the **center between the two classes** therefore inducing an **equal likelihood to misclassify points from either class**.  

Here is an example of an optimal decision surface,

<center>
<img 
src="https://gamedevacademy.org/wp-content/uploads/2017/10/SVM-Decision-Boundary.png.webp"
width="350"
height="300">
</center>

The area around the dotted line is called the **margin**.

**Observation**: Decision surface with large margins have a lower likelihood of misclassification because the class distributions are further away from the decision surface.

[source](https://gamedevacademy.org/classification-with-support-vector-machines/)

# Support Vector Machines

Support vector machines (SVMs) compute the maximum (optimal) margin between two classes and then place the decision surface at the center of the margin.

In [ ]:
X,y = make_data()

In [ ]:
svm_model = SVC(kernel='linear').fit(X, y)


In [ ]:
plot_data(X,y)
plot_svm_surface(svm_model, X);

As you can see from the figure the margin is limited by points from the two classes.  These limiting points are called **support vectors**.  One way to think about these points is that they **support** the position of the optimal decision surface essentially act as **constraints** on the width and position of the optimal decision surface.

The fact we are using a **linear kernel** in our example means that we are constructing the margin and the decision surface in training data space.  If we use kernels other than the linear kernels we map our training data into  other, extremely high-dimensional spaces and build our SVMs there.  This gives rise to non-linear decision surfaces when we map the decision surface from the high-dimensional space to training data space.

## Comparing SVMs and Perceptrons

We can compare the two algorithms side-by-side.

In [ ]:
pmodel = Perceptron().fit(X,y)

Plotting the two decision surfaces.

In [ ]:
plot_data(X,y)
plot_svm_surface(svm_model, X, surface_only=False);
plot_perceptron_surface(pmodel,X)

The decision surface for the SVM is given in black and the decision surface for the perceptron is given in green.  We can clearly see that the decision surface for the SVM is what we consider optimal.

## Dealing with Noisy Data - Soft Margins

Let regenerate our original data...

In [ ]:
X, y = make_data()

but we will introduce a noisy point...

In [ ]:
y[-5] = 0 # introducing noise by mapping a 1 class point to the 0 class

In [ ]:
plot_data(X,y)

Notice, there is one red point in the "wrong" class distribution.  Now let's fit an SVM with a hard margin.  Note that

* margin_hardness = 1 is considered a soft margin (more on that further down)
* margin_hardness > 100 is considered a hard margin

In [ ]:
margin_hardness = 1000  # very hard margin

The parameter C controls the margin, here we assign it a hardness value of 1,000.

In [ ]:
model = SVC(kernel='linear', C=margin_hardness).fit(X, y)

plot_data(X,y)
plot_svm_surface(model, X);

Notice that because of that one noisy point we obtain a suboptimal decision surface; a surface very close to the yellow class meaning that has a high likelihood to misclassify yellow points.

Now, let's set the margin hardness to 1 meaning that the margin construction is "soft" and and can ignore noisy points.

In [ ]:
margin_hardness = 1

Again, we set the C parameter to our margin hardness for fitting,

In [ ]:
model = SVC(kernel='linear', C=margin_hardness).fit(X, y)

plot_data(X,y)
plot_svm_surface(model, X);

Notice that we regained a margin that is optimal or close to optimal.  Also notice that the noisy point is on the wrong side of the decision surface.  In essence, the margin construction ingnored the noisy point in favor of a more optimal decision surface.  Exactly the kind of result we would like to see in noisy data.

Note that the default for SVMs is `C=1`, that is, by default the construction uses soft margins.

## Nonlinear Data

Not all data is separable by a linear decision surface.  For example, in the extreme case where one class is embedded in another class, like so,

In [ ]:
X, y = make_circles(100, factor=.1, noise=.1)

plot_data(X,y)

Here we can use **kernels** to model the non-linearities of the data.

Let's use the **radial basis function (rbf)** kernel.  We will also use a **hard margin** because we know that the two classes are well separated.



In [ ]:
model = SVC(kernel='rbf', C=1000).fit(X, y)

Let's visualize this decision surface in our training data space.

In [ ]:
plot_data(X,y)
plot_svm_surface(model,X)


We can see the non-linear decision surface with its support vectors

# Example: Text Classification

Because the SVM algorithm does not deteriorate with high dimensional data we can try it on our text classification problem.

In [ ]:
# get the newsgroup database
newsgroups = pd.read_csv(home+"newsgroups-noheaders.csv")
newsgroups.head(n=10)

In [ ]:
docterm = dsutils.docterm_matrix(newsgroups['text'],
                                 stop_words='english',
                                 min_df=2,
                                 stem=True,
                                 token_pattern="[a-zA-Z]+")
docterm

In [ ]:
# set up train and test sets
X_train, X_test, y_train, y_test = \
  train_test_split(docterm,   # as X
                   newsgroups['label'],  # as y
                   train_size=0.8,
                   test_size=0.2,
                   random_state=2)

In [ ]:
# model object
model = SVC()

In [ ]:
# grid search
param_grid = {
      'kernel':['linear','rbf'],
      'C': [1, 10, 100, 1000],
      'gamma': [0.0001, 0.001, 0.01]
}
grid = GridSearchCV(model, param_grid).fit(X_train, y_train)
print("Grid Search: best parameters: {}".format(grid.best_params_))
best_model = grid.best_estimator_

In [ ]:
dsutils.acc_score(best_model, X_test, y_test, as_string=True)

In [ ]:
dsutils.confusion_matrix(best_model, X_test, y_test)

Putting this into context with all the other classifiers we tried on this dataset:

**decision tree**: Accuracy: 0.74 (0.68,0.80)

**SVM**: Accuracy: 0.87 (0.82,0.92)

**gradient boosting**: Accuracy: 0.91 (0.87,0.95)

**random forest**: Accuracy: 0.92 (0.89, 0.96)

**naive bayes**: Accuracy: 0.96 (0.93,0.98)


# Example: Handwritten Digit Classification

Our handwritten digit information lives in 64 dimensional space.  Again, very suitable for the SVM algorithm.

In [ ]:
# fetch dataset
digits = pd.read_csv(home+'optdigits.csv', header=None)
digits.columns = ['a'+str(i) for i in range(1,65)] + ['digit']
digits

In [ ]:
# data
X = digits.drop(columns=['digit'])
y = digits[['digit']]

Note that this is a 10-way classification problem.  A single SVM can only discriminate two classes. The sklearn SVM implementation deals with it by building one SVM for each class (digit in this case) and then uses an aggregation scheme to come up with a single classification.

In [ ]:
# setting up training/testing data
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y['digit'], # we want a series as target
    train_size=0.8,
    test_size=0.2,
    random_state=1
)

In [ ]:
# model object
model = SVC()


In [ ]:
# grid search
param_grid = {
      'kernel':['linear','rbf'],
      'C': [1, 10, 100, 1000],
      'gamma': [0.0001, 0.001, 0.01, 0.1]
}
grid = GridSearchCV(model, param_grid, cv=3).fit(X_train, y_train)
print("Grid Search: best parameters: {}".format(grid.best_params_))
best_model = grid.best_estimator_

In [ ]:
dsutils.acc_score(best_model, X_test, y_test, as_string=True)

In [ ]:
dsutils.confusion_matrix(best_model, X_test, y_test)

Putting the SVM results in the context of our previous results,

**decision tree**: Accuracy: 0.92 (0.90, 0.93)

**random forest**: Accuracy: 0.97 (0.96, 0.98)

**gradient boosting**: Accuracy: 0.99 (0.98, 0.99)

**SVM**: Accuracy: 0.99 (0.99, 1.00)



# Reading

[Support Vector Machines](https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html)